In [2]:
# import necessary Libraries 
import pandas as pd

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'bangla-newspaper-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F258862%2F1576225%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240627%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240627T185144Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D76609c0a511815ba9dabeb25a8c793bebc796969a077dff5358f1d8218f580397adff44f75999d02fa423cbaf0eef8b57ee2a36c127e4dd48ec85b4564aa82d672f658d7918b002f451ac93f609e55ebf2b449fe1ef0a2d9c417115eabe17b976da2c7cca6d6add954b768cece36fa410d3aff206b2c5486eb939148cc689eff1eb36e4fa98ed44fe92ec5ce7af162c47d753bfd17834ddc30ffc87236c2b743d710d5deab6ff1a8c6d4e94fb90005dc3d98354546537f49b076feca729c9b37f72a59bfc5d4a526e95789814b1518ba6ea0808f1633f818469c3693ef10bc138ed0a160495c09b4506ac4cba7202c1b14b1e1dcfab6a4de51c1442d4c407547'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [3]:
df=pd.read_json("/kaggle/input/bangla-newspaper-dataset/data/data.json")

In [4]:
df.head(2)

,author,category,category_bn,published_date,modification_date,tag,comment_count,title,url,content
0,গাজীপুর প্রতিনিধি,bangladesh,বাংলাদেশ,"০৪ জুলাই ২০১৩, ২৩:২৬","০৪ জুলাই ২০১৩, ২৩:২৭",[গাজীপুর],0,"কালিয়াকৈরে টিফিন খেয়ে ৫০০ শ্রমিক অসুস্থ, বিক...",http://www.prothom-alo.com/bangladesh/article/...,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...
1,অনলাইন ডেস্ক,sports,খেলা,"০৪ জুলাই ২০১৩, ২৩:০৯","০৪ জুলাই ২০১৩, ২৩:১১",[টেনিস],0,সেমিফাইনাল বাধাও পেরিয়ে গেলেন লিসিকি,http://www.prothom-alo.com/sports/article/19028,এবারের উইম্বলডনটা স্মরণীয় করে রাখার মিশনেই যে...


In [5]:
df = df[['content']]


In [6]:
# Increase display options
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_colwidth', None)  # Display full width of each column
pd.set_option('display.max_rows', None)  # Display all rows if needed
df.head(1)

,content
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...


In [7]:
# Combine the content of the column into a single variable, separated by newline character
combined_content = '\n'.join(df['content'].astype(str).tolist())


In [8]:
combined_content[0:100]

'গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায় আজ বৃহস্পতিবার রাতের টিফিন খেয়ে একটি পোশাক কারখানার '

In [9]:
print("length of dataset1 in characters: ", len(combined_content))

length of dataset1 in characters:  820065032


In [10]:
text=combined_content

In [12]:
output_path="/kaggle/working/full.txt"
# Opening the file in 'w' mode (write mode) instead of 'wb' (binary write mode)
with open(output_path, "w") as outfile:
    outfile.write(text)
    print(f"Successfully wrote content into {output_path}")


Successfully wrote content into /kaggle/working/full.txt


In [14]:
import tokenizers
from tokenizers import Tokenizer
corpus_file = "/kaggle/working/full.txt"
tokenizer = tokenizers.SentencePieceBPETokenizer()
tokenizer.train(corpus_file)
tokenizer.save("Bengali_bpe_tokenizer.json")

In [17]:
tokenizer = Tokenizer.from_file("Bengali_bpe_tokenizer.json")

def compression_ratio(text, tokenizer):
    encoded = tokenizer.encode(text)
    compressed_length = len(encoded.ids)
    original_length = len(text)
    return original_length / compressed_length

sample_text =  "গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়..."
encoded_text = tokenizer.encode(sample_text)
compression = compression_ratio(sample_text, tokenizer)

print(f"Encoded text: {encoded_text.tokens}")
print(f"Vocabulary size: {len(tokenizer.get_vocab())}")
print(f"Compression ratio: {compression}")
print(f"Decoded text: {tokenizer.decode(encoded_text.ids)}")

Encoded text: ['▁গাজীপুরের', '▁কালিয়াকৈর', '▁উপজেলার', '▁তেল', 'ির', 'চালা', '▁এলাকায়', '...']
Vocabulary size: 30000
Compression ratio: 6.125
Decoded text: গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...
